In [ ]:
import os
# os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import open3d as o3d
from scipy.spatial import cKDTree
import torch
from PIL import Image
from tqdm import tqdm

## 计算 Clip 相似度

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

model.to("cuda")

In [ ]:
def load_and_preprocess_image(image_path):
    # Load the image from the specified path
    image = Image.open(image_path)
    # Apply the CLIP preprocessing to the image
    image = processor(images=image)
    # Return the preprocessed image
    return image


def clip_img_score (img1_path,img2_path):
    # Load the two images and preprocess them for CLIP
    image_a = load_and_preprocess_image(img1_path)["pixel_values"]
    image_b = load_and_preprocess_image(img2_path)["pixel_values"]

    # Calculate the embeddings for the images using the CLIP model
    with torch.no_grad():
        embedding_a = model.get_image_features(torch.tensor(np.array(image_a)).to("cuda"))
        embedding_b = model.get_image_features(torch.tensor(np.array(image_b)).to("cuda"))

    # Calculate the cosine similarity between the embeddings
    similarity_score = torch.nn.functional.cosine_similarity(embedding_a, embedding_b)
    return similarity_score.item()

In [ ]:
res = {}
for model_name in tqdm(os.listdir("/home/ljr/Machine-Learning-Project/val/gt")):
    print(model_name)
    res[model_name] = [0.0, 0.0]
    for i in range(8):
        for j in range(4):
            res[model_name][0] += clip_img_score(f"/home/ljr/Machine-Learning-Project/val/gt/{model_name}/view_{i}_{j}_10.png", f"/home/ljr/Machine-Learning-Project/val/val_finetune_rendered/{model_name}/view_{i}_{j}_10.png")
            res[model_name][1] += clip_img_score(f"/home/ljr/Machine-Learning-Project/val/gt/{model_name}/view_{i}_{j}_10.png", f"/home/ljr/Machine-Learning-Project/val/val_zeroshot_rendered/{model_name}/view_{i}_{j}_10.png")
    res[model_name][0] /= 32
    res[model_name][1] /= 32
    print(res[model_name])

# 将结果写入txt文件
with open('clip_scores.txt', 'w') as f:
    for model_name, scores in res.items():
        f.write(f"{model_name}: \nfinetune={scores[0]:.5f}, zeroshot={scores[1]:.5f}\n")


In [ ]:
# 将结果转换为DataFrame格式
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(res).T
df.columns = ['finetune', 'zeroshot']

# 设置图形大小
plt.figure(figsize=(15, 6), dpi=300)

# 绘制直方图
x = np.arange(len(df.index))
width = 0.35

plt.bar(x - width/2, df['finetune'], width, label='Finetune')
plt.bar(x + width/2, df['zeroshot'], width, label='Zeroshot')

# 设置图形属性
plt.xlabel('model name')
plt.ylabel('CLIP score')
plt.title('Finetune vs Zeroshot CLIP score')
plt.xticks(x, df.index, rotation=45, ha='right', fontsize=8)
plt.legend()

# 调整布局避免标签重叠
plt.tight_layout()

# 显示图形
plt.show()

# 计算finetune和zeroshot的平均值
finetune_mean = df['finetune'].mean()
zeroshot_mean = df['zeroshot'].mean()

print(f"Finetune平均值: {finetune_mean:.5f}")
print(f"Zeroshot平均值: {zeroshot_mean:.5f}")



In [ ]:
# 读取数据
with open('/home/ljr/Machine-Learning-Project/clip_scores_1.txt', 'r') as f:
    lines = f.readlines()

# 解析数据
finetune_scores = []
zeroshot_scores = []
for line in lines:
    if 'finetune' in line:
        parts = line.split(': ')
        scores = parts[1].split(', ')
        finetune_scores.append(float(scores[0].split('=')[1]))
        zeroshot_scores.append(float(scores[1].split('=')[1]))

# 创建对比图
plt.figure(figsize=(12, 6), dpi=100)
plt.plot(finetune_scores, label='Finetune', marker='o', color='#1f77b4', linewidth=2, markersize=6)
plt.plot(zeroshot_scores, label='Zeroshot', marker='x', color='#ff7f0e', linewidth=2, markersize=6)

# 设置图形属性
plt.xlabel('Sample Index', fontsize=12)
plt.ylabel('CLIP Score', fontsize=12)
plt.title('CLIP Score Comparison: Finetune vs Zeroshot', fontsize=14, pad=20)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, linestyle='--', alpha=0.6)

# 调整刻度字体
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# 调整布局
plt.tight_layout()
plt.show()

## 计算 F-score


In [ ]:
import numpy as np
import open3d as o3d
import trimesh
from scipy.spatial import cKDTree
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

def load_mesh(file_path):
    """加载3D模型文件（支持glb和obj格式）"""
    if file_path.endswith('.glb'):
        scene = trimesh.load(file_path)
        if isinstance(scene, trimesh.Scene):
            meshes = []
            for g in scene.geometry.values():
                if isinstance(g, trimesh.Trimesh):
                    meshes.append(g)
                elif hasattr(g, 'vertices') and hasattr(g, 'faces'):
                    meshes.append(trimesh.Trimesh(vertices=g.vertices, faces=g.faces))
            
            if not meshes:
                raise ValueError(f"No valid mesh found in {file_path}")
            mesh = trimesh.util.concatenate(meshes)
        else:
            mesh = scene
            
        vertices = np.array(mesh.vertices)
        faces = np.array(mesh.faces)
        o3d_mesh = o3d.geometry.TriangleMesh()
        o3d_mesh.vertices = o3d.utility.Vector3dVector(vertices)
        o3d_mesh.triangles = o3d.utility.Vector3iVector(faces)
    else:
        o3d_mesh = o3d.io.read_triangle_mesh(file_path)
    
    if not o3d_mesh.has_vertices():
        raise ValueError(f"Loaded mesh has no vertices: {file_path}")
    
    return o3d_mesh

def normalize_points(points):
    """归一化点云到单位球内"""
    centroid = np.mean(points, axis=0)
    points = points - centroid
    max_dist = np.max(np.sqrt(np.sum(points ** 2, axis=1)))
    points = points / max_dist
    return points

def align_point_clouds(pred_points, gt_points):
    """对齐两个点云"""
    pred_points = normalize_points(pred_points)
    gt_points = normalize_points(gt_points)
    
    pred_pcd = o3d.geometry.PointCloud()
    pred_pcd.points = o3d.utility.Vector3dVector(pred_points)
    gt_pcd = o3d.geometry.PointCloud()
    gt_pcd.points = o3d.utility.Vector3dVector(gt_points)
    
    pred_pcd.estimate_normals()
    gt_pcd.estimate_normals()
    
    reg = o3d.pipelines.registration.registration_icp(
        pred_pcd, gt_pcd,
        max_correspondence_distance=0.1,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=50)
    )
    
    pred_aligned = pred_pcd.transform(reg.transformation)
    return np.asarray(pred_aligned.points)

def compute_f_score(pred_points, gt_points, threshold=0.05):
    """计算F-score"""
    pred_tree = cKDTree(pred_points)
    gt_tree = cKDTree(gt_points)
    
    pred_to_gt_dist, _ = gt_tree.query(pred_points)
    gt_to_pred_dist, _ = pred_tree.query(gt_points)
    
    precision = np.mean(pred_to_gt_dist < threshold)
    recall = np.mean(gt_to_pred_dist < threshold)
    f_score = 2 * precision * recall / (precision + recall + 1e-8)
    
    return precision, recall, f_score

def compute_fscore(gt_path, pred_path, n_points=5000):
    """计算两个3D模型之间的F-score"""
    try:
        gt_mesh = load_mesh(gt_path)
        pred_mesh = load_mesh(pred_path)
        
        gt_pcd = gt_mesh.sample_points_uniformly(n_points)
        pred_pcd = pred_mesh.sample_points_uniformly(n_points)
        
        gt_points = np.asarray(gt_pcd.points)
        pred_points = np.asarray(pred_pcd.points)
        
        pred_aligned = align_point_clouds(pred_points, gt_points)
        
        precision, recall, f_score = compute_f_score(pred_aligned, gt_points, threshold=0.05)
        return precision, recall, f_score
    except Exception as e:
        print(f"Error processing {gt_path} and {pred_path}: {str(e)}")
        return None

if __name__ == "__main__":
    results = {'finetune': [], 'zeroshot': []}
    
    for model_name in tqdm(os.listdir("/home/ljr/Machine-Learning-Project/val/gt")):
        gt_path = f"/home/ljr/Machine-Learning-Project/needed_models/{model_name}/{model_name}.glb"
        
        if not os.path.exists(gt_path):
            print(f"Skipping {model_name}: GT file not found")
            continue
            
        pred_paths = {
            'finetune': f"/home/ljr/Machine-Learning-Project/val/val_finetune_rendered/{model_name}/mesh.obj",
            'zeroshot': f"/home/ljr/Machine-Learning-Project/val/val_zeroshot_rendered/{model_name}/mesh.obj"
        }
        
        print(f"\n处理模型: {model_name}")
        
        for method, pred_path in pred_paths.items():
            if not os.path.exists(pred_path):
                print(f"Skipping {method} for {model_name}: File not found")
                continue
                
            scores = compute_fscore(gt_path, pred_path)
            if scores:
                precision, recall, f_score = scores
                results[method].append({
                    'model': model_name,
                    'precision': precision,
                    'recall': recall,
                    'f_score': f_score
                })
                
                print(f"\n{method}评估结果:")
                print(f"F-score: {f_score:.4f}")
                print(f"Precision: {precision:.4f}")
                print(f"Recall: {recall:.4f}")
    
    # 计算平均值
    for method in ['finetune', 'zeroshot']:
        if results[method]:
            avg_f_score = np.mean([r['f_score'] for r in results[method]])
            avg_precision = np.mean([r['precision'] for r in results[method]])
            avg_recall = np.mean([r['recall'] for r in results[method]])
            
            print(f"\n{method}平均结果:")
            print(f"Average F-score: {avg_f_score:.4f}")
            print(f"Average Precision: {avg_precision:.4f}")
            print(f"Average Recall: {avg_recall:.4f}")

    # 将结果保存到文件
    with open('fscore_results.txt', 'w') as f:
        for method in ['finetune', 'zeroshot']:
            f.write(f"\n{method}结果:\n")
            for result in results[method]:
                f.write(f"{result['model']}:\n")
                f.write(f"F-score: {result['f_score']:.4f}\n")
                f.write(f"Precision: {result['precision']:.4f}\n")
                f.write(f"Recall: {result['recall']:.4f}\n")
                f.write("-" * 50 + "\n")

In [ ]:
# 读取结果文件并解析数据
import matplotlib.pyplot as plt

def parse_results_file(filename):
    results = {'finetune': [], 'zeroshot': []}
    current_method = None
    current_model = None
    
    with open(filename, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        if line.endswith('结果:'):
            current_method = line.split('结果')[0]
            continue
            
        if line.endswith(':'):
            current_model = line[:-1]
            continue
            
        if line.startswith('F-score:'):
            f_score = float(line.split(':')[1])
            results[current_method].append({
                'model': current_model,
                'f_score': f_score
            })
            
    return results

# 解析结果
results = parse_results_file('fscore_results.txt')

# 准备绘图数据
finetune_models = [r['model'] for r in results['finetune']]
finetune_scores = [r['f_score'] for r in results['finetune']]
zeroshot_models = [r['model'] for r in results['zeroshot']]
zeroshot_scores = [r['f_score'] for r in results['zeroshot']]

# 创建柱状图
plt.figure(figsize=(12, 6))
x = range(len(finetune_models))
width = 0.35

plt.bar([i - width/2 for i in x], finetune_scores, width, label='微调', color='skyblue')
plt.bar([i + width/2 for i in x], zeroshot_scores, width, label='零样本', color='lightcoral')

plt.xlabel('模型')
plt.ylabel('F-score')
plt.title('微调 vs 零样本 F-score 对比')
plt.xticks(x, finetune_models, rotation=45)
plt.legend()

plt.tight_layout()
plt.show()
